In [29]:

construction_type = "charcoal_hearth_hill"
cfg_name = 'cfg20200624T1134'
model_epoch='0016'

print("cfg_name:",cfg_name)
#import pickle
import os
from os import listdir
import cv2
#with open('rcnn_100_random_images_ferebee.pkl', 'rb') as f:
#    test_file_names = pickle.load(f)

image_prediction_folder = '/storage/images/'+construction_type+'/predictions/'+cfg_name+"/unknown/"
image_for_prediction_folder = '/storage/images/'+construction_type+'/images_training/'
training_annots_folder = '/storage/images/'+construction_type+'/annots/'

test_file_names = []
for filename in listdir(image_for_prediction_folder):
    test_file_names.append(filename)

# display image with masks and bounding boxes
from os import listdir
import skimage
import random
    from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# set up font for boxes
# https://www.geeksforgeeks.org/python-opencv-cv2-puttext-method/
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
font_scale = 1
font_color = (255, 255, 255) 
font_thickness = 2
   

import cv2


cn=0
while cn<len(test_file_names):
    
    print("jeff",cn)
    image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
    print(image.shape)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    #https://stackoverflow.com/questions/32609098/how-to-fast-change-image-brightness-with-python-opencv
    #make it brighter
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v += 42
    final_hsv = cv2.merge((h, s, v))

    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

    #value = 42 #whatever value you want to add
    #cv2.add(hsv[:,:,2], value, hsv[:,:,2])
    #image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    obj_f_num = test_file_names[cn]
    obj_f_num = obj_f_num[:6]
    print(cn,obj_f_num)
    overlay = image.copy()
    output = image.copy()
    
    #Get original training data too
    tree_train = ElementTree.parse(training_annots_folder+obj_f_num+'.xml')
    print(training_annots_folder+obj_f_num+'.xml')
    #print(tree_train)
    # get the root of the document
    root_train = tree_train.getroot()
    for obj in root_train.findall('./object'):
        box = obj.find('bndbox')
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)

        cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(255, 255, 255), 6)
        #cv2.putText(output, '['+box_num+'] S: '+ score[:5], (xmin+10, ymin+70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
    
    
    cv2.putText(output, str(cn), (10,30), font, font_scale, font_color, font_thickness, cv2.LINE_AA)     
    tree = ElementTree.parse(image_prediction_folder+obj_f_num+'.xml')
    print(image_prediction_folder+obj_f_num+'.xml')
    #print(tree)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    #object_present = root.find('./object_present').text
    object_present="Yes"
    #print(fn_image)
    box_num=0

    boxes_correct={}
    for obj in root.findall('./object'):
        score = obj.find('score').text
        correct = obj.find('correct').text
        if(correct is None):
            correct="0"
        if(correct==""):
            correct="0"
        correct=int(correct)
        box = obj.find('bndbox')
        box_num=obj.find('number').text
        boxes_correct[str(box_num)] = correct
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)
        if(correct==0):
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(0, 0, 255), 2)
        elif(correct==1):
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(0, 255,0 ), 2)
        # Using cv2.putText() method 
        #cv2.putText(output, '['+box_num+'] S: '+ score[:5], (xmin+10, ymin+70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
        
        if (int(box_num)<=25):
            box_keys=chr(65+32+int(box_num))
        if (int(box_num)>=26 and int(box_num)<=35):
            box_keys=chr(48+int(box_num)-26)
        if (int(box_num)>=36 and int(box_num)<=45):
            box_keys=chr(32+int(box_num)-36)
            
        if (ymin+80)<image.shape[0]:
            cv2.putText(output, box_keys , (xmin+10, ymin+70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
        else:
            cv2.putText(output, box_keys , (xmin+10, ymin-10), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
    
    
        
    alpha = .8
    #cv2.addWeighted(overlay, alpha, output, 1-alpha,0, output)
    #cv2.putText(output, '['+str(cn)+'] Present: '+ object_present, (50, 70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
    image = output
    scale_percent = 1024/image.shape[1] # percent of original size
    #print(scale_percent)
    width = int(image.shape[1] * scale_percent )
    height = int(image.shape[0] * scale_percent )
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized_original = resized.copy()
    
    object_file_num_int_return = -1
 # keep looping until the 'c' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", resized)
        cv2.moveWindow("image", 10,10)
        #key = cv2.waitKey(1) & 0xFF
        key = cv2.waitKey()

        # if the '!' key is pressed, break from the loop
        
            
        if key == ord("/"):
            object_file_num_int_return = -1
            break
        if key == ord("+"):
            break
        if key == ord("-"):
            cn = cn-2
            break
        if key == ord(">"):
            cn = cn+9
            break
        if key == ord("?"):
            cn = cn+99
            break
           
        if key == ord("~"):
            break
        if key>=ord("a") and key<=ord("z"):
            bc_key = key-97
            if(bc_key<len(boxes_correct)):
                print("                                                           ","low key ord",key,bc_key,boxes_correct[str(bc_key)],str(int(not int(boxes_correct[str(bc_key)]))))
                boxes_correct[str(bc_key)]=str(int(not int(boxes_correct[str(bc_key)])))

        if key>=ord("0") and key<=ord("9"):
            bc_key = key-48+26
            if(bc_key<len(boxes_correct)):
                print("                                                           ","key ord",key,key-48,bc_key,boxes_correct[str(bc_key)],str(int(not int(boxes_correct[str(bc_key)]))))
                boxes_correct[str(bc_key)]=str(int(not int(boxes_correct[str(bc_key)])))

        if key>=ord(" ") and key<=ord(")"):
            bc_key = key-32+36
            if(bc_key<len(boxes_correct)):
                print("                                                           ","key ord",key,key-32,bc_key,boxes_correct[str(bc_key)],str(int(not int(boxes_correct[str(bc_key)]))))
                boxes_correct[str(bc_key)]=str(int(not int(boxes_correct[str(bc_key)])))
          

    if object_file_num_int_return == -1:
        #correct_number = root.find('./correct_number')
        for obj in root.findall('./object'):
            box_num=obj.find('number').text
            correct = obj.find('correct')
            correct.text = str(boxes_correct[str(box_num)])
            #print("                                                           ",box_num, boxes_correct[str(box_num)],correct.text)
            #print(boxes_correct[str(box_num)])
        #correct_number.text = str(object_file_num_int_return)
        #root.set('correct_number',str(object_file_num_int_return))
        
        #op = root.find('object_present')  #.text = "xx" #object_present
        #op.text = str(object_present)
        tree.write(image_prediction_folder+obj_f_num+'.xml')
        #write out an annot file of just the correct boxes
        tree_edited = tree
        root_edited = tree_edited.getroot()
        for obj in root_edited.findall('./object'):
            box_num=obj.find('number').text
            correct = obj.find('correct')
            if(correct.text=="0"):
                print("removed",obj)
                root.remove(obj)
        tree_edited.write(image_prediction_folder+"/edited/"+obj_f_num+'.xml')
            
            
    # close all open windows
    cv2.destroyAllWindows()
    image=None
    if key == ord("~"):
        break
    cn=cn+1

cfg_name: cfg20200624T1134
jeff 0
(768, 1024)
0 581006
/storage/images/charcoal_hearth_hill/annots/581006.xml
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/581006.xml
jeff 100
(768, 1024)
100 461009
/storage/images/charcoal_hearth_hill/annots/461009.xml
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/461009.xml
removed <Element 'object' at 0x7f5dea940138>
jeff 200
(768, 1024)
200 391508
/storage/images/charcoal_hearth_hill/annots/391508.xml
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/391508.xml
removed <Element 'object' at 0x7f5dea972278>
jeff 210
(768, 1024)
210 430402
/storage/images/charcoal_hearth_hill/annots/430402.xml
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/430402.xml
removed <Element 'object' at 0x7f5dea941958>
removed <Element 'object' at 0x7f5dea970188>
removed <Element 'object' at 0x7f5dea99bea8>
removed <Element 'object' at 0x7f5dea94f368>
jeff 220
(768, 1024

In [13]:
#This is a onetime tfix to xml files to exchange width with heigh

from os import listdir
import os
from xml.etree import ElementTree
#with open('rcnn_100_random_images_ferebee.pkl', 'rb') as f:
#    test_file_names = pickle.load(f)

edited_annot_folder = '/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/'

test_file_names = []
for filename in listdir(edited_annot_folder):
    test_path = os.path.join(edited_annot_folder, filename)
    if os.path.isdir(test_path):
        # skip directories
        continue
    else:
        test_file_names.append(filename)
cn=0
while cn<len(test_file_names):
    
    obj_f_num = test_file_names[cn]
    obj_f_num = obj_f_num[:6]
    print(cn,obj_f_num)
    tree_edited = ElementTree.parse(edited_annot_folder+obj_f_num+'.xml')
    print(edited_annot_folder+obj_f_num+'.xml')

    # get the root of the document
    root_edited = tree_edited.getroot()
    # extract each bounding box
    width_tag = root_edited.find('./size/width')
    height_tag = root_edited.find('./size/height')
    width_old = root_edited.find('./size/width').text
    height_old = root_edited.find('./size/height').text
    width_tag.text = height_old
    height_tag.text = width_old
    print("width_old",width_old,"height_old",height_old, "width_tag.text",width_tag.text,"height_tag.text",height_tag.text)

    tree_edited.write(edited_annot_folder+"fixed/"+obj_f_num+'.xml')
    cn=cn+1

0 520303
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/520303.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
1 430707
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/430707.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
2 430506
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/430506.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
3 461109
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/461109.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
4 411905
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/411905.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
5 581106
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/581106.xml
width_old 768 height_old 1024 width_tag.text

width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
66 460700
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/460700.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
67 460604
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/460604.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
68 520104
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/520104.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
69 461012
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/461012.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
70 520203
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/520203.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
71 391107
/storage/images/charcoal_hearth_hill/predictions/cfg20200624

width_old 697 height_old 1024 width_tag.text 1024 height_tag.text 697
149 411104
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/411104.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
150 430108
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/430108.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
151 580811
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/580811.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
152 411307
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/411307.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
153 460602
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/460602.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
154 461510
/storage/images/charcoal_hearth_hill/predictions/cfg20

width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
243 460600
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/460600.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
244 520701
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/520701.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
245 430704
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/430704.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
246 581111
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/581111.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
247 460601
/storage/images/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/edited/460601.xml
width_old 768 height_old 1024 width_tag.text 1024 height_tag.text 768
248 410607
/storage/images/charcoal_hearth_hill/predictions/cfg20

In [8]:
print(ord("a"))

cv2.imshow("image", resized)
cv2.moveWindow("image", 10,10)
key = cv2.waitKey(1) & 0xFF
print(key)
cv2.destroyAllWindows()
image=None

97
255


In [6]:
bc_key=10
print(boxes_correct)

{'0': 1, '1': '0', '2': '0', '3': '0', '4': '0', '5': '0', '6': '0', '7': '0', '8': '0', '9': '0'}


In [14]:
print(int(not int("0")))

1


# Gather the data from the xml files into results_cfg_.csv

In [44]:

for x in range (0,32):
    print(x,x+32,chr(32+x) )

0 32  
1 33 !
2 34 "
3 35 #
4 36 $
5 37 %
6 38 &
7 39 '
8 40 (
9 41 )
10 42 *
11 43 +
12 44 ,
13 45 -
14 46 .
15 47 /
16 48 0
17 49 1
18 50 2
19 51 3
20 52 4
21 53 5
22 54 6
23 55 7
24 56 8
25 57 9
26 58 :
27 59 ;
28 60 <
29 61 =
30 62 >
31 63 ?


In [32]:
print("cfg_name:",cfg_name)
image_prediction_folder = '/home/student/data_5000_project/data/images/construction_types/processed/'+construction_type+'/predictions/'+cfg_name+"/unknown/"

# display image with masks and bounding boxes
from os import listdir
from xml.etree import ElementTree

results = open(image_prediction_folder+"results_"+cfg_name+".csv","w+")
results.write("min_score,gt_obj_pres_pos,gt_obj_pres_neg,tp,fp,tn,fn,tp_box,fp_box\n")
min_scores = []
min_scores.append(.7)
min_scores.append(.75)
min_scores.append(.80)
min_scores.append(.85)
min_scores.append(.90)
min_scores.append(.95)
for min_score in min_scores:
    gt_obj_pres_pos = 0
    gt_obj_pres_neg = 0
    tp=0
    fp=0
    tn=0
    fn=0
    tp_box=0
    fp_box=0

    

    for cn in range(0,100):
    
        obj_f_num = "00000"+str(cn)
        obj_f_num = obj_f_num[-5:]
        #print(fn)
        tree = ElementTree.parse(image_prediction_folder+obj_f_num+'.xml')
        print(image_prediction_folder+obj_f_num+'.xml')
        print(tree)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
    

    
        #print(fn_image)
        box_num=0
        boxes_correct={}
        obj_pres_pred = 0
        for obj in root.findall('./object'):
            score = obj.find('score').text
            score = float(score)
            if score>=min_score:
                correct = obj.find('correct').text
                if(correct is None):
                    correct="0"
                if(correct==""):
                    correct="0"
                correct=int(correct)
                if(correct==0):
                    fp_box=fp_box+1
                if(correct==1):
                    tp_box=tp_box+1
                #if one is correct, it's deemed found in the picture.
                obj_pres_pred = 1
                #obj_pres_pred or correct
                print(correct,obj_pres_pred)
        object_present = root.find('./object_present').text
        if(object_present=="1"):
            gt_obj_pres_pos = gt_obj_pres_pos + 1
            if(obj_pres_pred==1):
                tp=tp+1
            if(obj_pres_pred==0):
                fn=fn+1
        if(object_present=="0"):
            gt_obj_pres_neg = gt_obj_pres_neg + 1
            if(obj_pres_pred==1):
                fp=fp+1
            if(obj_pres_pred==0):
                tn=tn+1
    print("gt_obj_pres_pos", gt_obj_pres_pos)
    print("gt_obj_pres_neg", gt_obj_pres_neg)
    #results.write("cfg_name,gt_obj_pres,gt_obj_not_pres,TP,FP,TN,FN\n")
    results.write(str(min_score)+","+str(gt_obj_pres_pos)+","+str(gt_obj_pres_neg)+","+str(tp)+","+str(fp)+","+str(tn)+","+str(fn)+","+str(tp_box)+","+str(fp_box)+"\n")

results.close()

cfg_name: cfg20200624T1134


FileNotFoundError: [Errno 2] No such file or directory: '/home/student/data_5000_project/data/images/construction_types/processed/charcoal_hearth_hill/predictions/cfg20200624T1134/unknown/00000.xml'

In [25]:
print(chr(65+32))

a
